In [48]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import LineString

import road

In [49]:
bbox =(48.7861400822045,2.197526708826558,48.93467868121917,2.4458831002388592)
bbox = (45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271)

highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link", 
                "secondary", "secondary_link", "tertiary", "tertiary_link", "residential"]
highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link"]

In [50]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query ="""
[out:json][timeout:180];
(
"""
overpass_query += ''.join([f'way["highway"="{highway}"]{bbox};\n' for highway in highway_list])
overpass_query +=""" 
);
out body;
>;
out skel qt;
"""

response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [51]:
{'overpass_query': overpass_query,'tags': ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']}

{'overpass_query': '\n[out:json][timeout:180];\n(\nway["highway"="motorway"](45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271);\nway["highway"="motorway_link"](45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271);\nway["highway"="trunk"](45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271);\nway["highway"="trunk_link"](45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271);\nway["highway"="primary"](45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271);\nway["highway"="primary_link"](45.516336095754156, -73.6492855702676, 45.644709880535515, -73.44752816469271);\n \n);\nout body;\n>;\nout skel qt;\n',
 'tags': ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']}

In [52]:
"""
[out:json][timeout:180][bbox:{{bbox}}];
(
way["highway"="residential"]; // find all nodes in current bounding box with start_date tag
);
convert way 
    ::id = id(),
	"nodes" = geometry, // Should be implemened in futur version
	"oneway" = t["oneway"];
out body;
>;
out skel qt;
"""

'\n[out:json][timeout:180][bbox:{{bbox}}];\n(\nway["highway"="residential"]; // find all nodes in current bounding box with start_date tag\n);\nconvert way \n    ::id = id(),\n\t"nodes" = geometry, // Should be implemened in futur version\n\t"oneway" = t["oneway"];\nout body;\n>;\nout skel qt;\n'

overpass_url = "http://overpass-api.de/api/interpreter"
data = []
for highway in highway_list:
    overpass_query ="""
    [out:json][timeout:180];
    (
    """
    overpass_query += ''.join(f'way["highway"={highway}]{bbox};\n')
    overpass_query +=""" 
    );
    out body;
    >;
    out skel qt;
    """
    data.append(overpass_query)


import asyncio
import aiohttp
import time

async def get(url, data, session):
    try:
        async with session.get(url=url, data=data) as response:
            resp = await response.json()
            print("Successfully got url {} with resp of length {}.".format(url, len(resp)))
            return await response
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(url, data):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, d, session) for d in data])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
    return ret

response = await main(overpass_url, data)

In [53]:
way = pd.DataFrame([d for d in data['elements'] if d['type'] == 'way']).set_index('id')
nodes = pd.DataFrame([d for d in data['elements'] if d['type'] == 'node']).set_index('id')

way_exploded = way.explode('nodes').merge(nodes[['lat','lon']], left_on='nodes', right_index=True, how='left')

geom = way_exploded.groupby('id')[['lon', 'lat']].apply(lambda x: LineString(x.values))
geom.name = 'geometry'
way = gpd.GeoDataFrame(way.join(geom), crs=4326)


/home/boivin/.pyenv/versions/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [54]:
tags = pd.DataFrame.from_records(way['tags'].values, index=way['tags'].index)
cols = ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']

In [55]:
way_tags = way.drop(columns=['nodes', 'tags'], errors='ignore').join(tags[cols])

In [56]:
# OSMNX add edges speed, length and time
# garder les liens fortement connexes
# retirer les noeuds de degree 2

In [57]:
# SOME CLEANING ON THE ONEWAY ... Work In Progress
way_tags['oneway'].fillna('no', inplace=True)
way_tags['oneway'] = way_tags['oneway'].replace('yes', True).replace('no', False).replace('-1', False).replace(-1, False).replace('alternating',False)


In [58]:
way_tags.to_file('way.geojson',driver='GeoJSON')

In [59]:
links, nodes = road.get_links_and_nodes('way.geojson', split_direction=False)
nodes = nodes.set_crs(links.crs)

In [60]:
print(len(way_tags))
print(len(links))

1566
1612


In [61]:
links = road.clean_maxspeed(links)

In [62]:
links = road.rectify_geometry_direction(links,nodes)

0 geometry to inverse


In [63]:
links = road.drop_duplicated_links(links)

0 links dropped


In [64]:
len(links)

1612

In [65]:
links = road.simplify(links)
len(links)

278 deg 2 nodes
find path with large cutoff for 15  origins
find path with large cutoff for 2  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring


947

In [66]:
links = road.split_oneway(links)
len(links)

1024

In [67]:
links = road.main_strongly_connected_component(links,None,False)
len(links)

347

In [68]:
def process_list_in_col(col_values,new_type,function):
    if isinstance(col_values, list):
        return  function([new_type(val) for val in col_values])
    else:
        return new_type(col_values)
    
def remove_list_in_col(col_values,method='first'):
    if isinstance(col_values, list):
        if method == 'first':
            return col_values[0]
        else:
            return col_values[-1]
        
    else:
        return col_values
    
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x,float,np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x,float,np.nanmean)).apply(lambda x: np.floor(x))

/tmp/ipykernel_363393/3507984463.py:3: RuntimeWarning: Mean of empty slice
  return  function([new_type(val) for val in col_values])


In [69]:
for col in ['id', 'type', 'highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))

In [70]:
links['length'] = links.to_crs(32618).length


In [71]:



    
links.loc[~links['maxspeed'].astype(str).str.isdigit(),'maxspeed']=np.nan
links['maxspeed'] = pd.to_numeric(links['maxspeed'])
speed_dict = links.dropna().groupby('highway')['maxspeed'].agg(np.mean).to_dict()
links.loc[~np.isfinite(links['maxspeed']),'maxspeed'] = links.loc[~np.isfinite(links['maxspeed']),'highway'].apply(lambda x: speed_dict.get(x))
links['time'] = links['length']/(links['maxspeed']*1000/3600)

In [72]:
links['lanes'] = pd.to_numeric(links['lanes'])
lane_dict = links.groupby('highway')['lanes'].agg(np.nanmean).apply(lambda x: np.floor(x)).to_dict()
links.loc[~np.isfinite(links['lanes']),'lanes'] = links.loc[~np.isfinite(links['lanes']),'highway'].apply(lambda x: lane_dict.get(x))


In [73]:
links = links.drop(columns=['id','type','oneway','surface'])
links.geometry = links.simplify(0.00003)

In [74]:
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)

nodes_set = set(links['a']).union(set(links['b']))

nodes = nodes.loc[nodes_set].sort_index()


/tmp/ipykernel_363393/722026835.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  nodes = nodes.loc[nodes_set].sort_index()


In [75]:
links = road.drop_duplicated_links(links)


0 links dropped


# Elevation

In [90]:
links.to_file('road_links.geojson')
nodes.to_file('road_nodes.geojson')